In [11]:
import os
import pandas as pd


In [12]:
game_pks_path = 'game_pks.csv'
gamelogs_dir = 'gamelogs/'
output_path = 'model/unsorted_masterdata.csv'

# Read the game_pks.csv file
game_pks_df = pd.read_csv(game_pks_path)
game_pks_list = game_pks_df['game_id'].tolist()

# Initialize an empty list to store DataFrames
dataframes = []

# Initialize a set to store all columns
all_columns = set()

# First pass: Collect all unique columns
for game_pk in game_pks_list:
    file_path = os.path.join(gamelogs_dir, f'gamestats_{game_pk}.csv')
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        all_columns.update(df.columns)
    else:
        print(f"BAD - File {file_path} not found.")

print("First pass: columns collected")

# Second pass: Read files and align columns
for game_pk in game_pks_list:
    file_path = os.path.join(gamelogs_dir, f'gamestats_{game_pk}.csv')
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        # Add missing columns with default value of NaN (handled by reindex)
        df = df.reindex(columns=all_columns)
        dataframes.append(df)

print("Second pass: data added.")

# Concatenate all the DataFrames
master_df = pd.concat(dataframes, ignore_index=True)

# Save the master DataFrame to a CSV file
master_df.to_csv(output_path, index=False)

print(f"Master dataset saved to {output_path}")

First pass: columns collected
Second pass: data added.
Master dataset saved to model/unsorted_masterdata.csv


### Sorting Dataset

In [13]:
# Load the dataset
df = pd.read_csv('model/unsorted_masterdata.csv', low_memory=False)

# Define a function to sort columns by player order and then alphabetically
def sort_columns(df):
    # List to store sorted column names
    sorted_columns = []
    
    # Ensure the specified order of the first few general columns
    first_columns = ['gamepk', 'game_date', 'runs_home', 'runs_away', 'runs_total']
    for col in first_columns:
        if col in df.columns:
            sorted_columns.append(col)
    
    # Lists to categorize columns
    away_batter_columns = [[] for _ in range(9)]
    home_batter_columns = [[] for _ in range(9)]
    away_pitcher_columns = [[] for _ in range(9)]
    home_pitcher_columns = [[] for _ in range(9)]
    
    # Helper function to ensure the list is long enough
    def ensure_length(lst, index):
        while len(lst) <= index:
            lst.append([])
    
    # Helper function to sort player-specific columns
    def sort_player_columns(columns):
        player_columns = []
        other_columns = []
        for col in columns:
            if any(key in col for key in ['Name', 'ID', 'bbrefID']):
                player_columns.append(col)
            else:
                other_columns.append(col)
        return sorted(player_columns) + sorted(other_columns)
    
    # Categorize columns
    for col in df.columns:
        if col.startswith('Away_Batter'):
            try:
                num = int(col.split('_')[1][6]) - 1
                ensure_length(away_batter_columns, num)
                away_batter_columns[num].append(col)
            except (ValueError, IndexError):
                continue
        elif col.startswith('Home_Batter'):
            try:
                num = int(col.split('_')[1][6]) - 1
                ensure_length(home_batter_columns, num)
                home_batter_columns[num].append(col)
            except (ValueError, IndexError):
                continue
        elif col.startswith('Away_P_') or col.startswith('Away_SP'):
            try:
                if 'SP' in col:
                    num = 0
                else:
                    num = int(col.split('_')[2])
                ensure_length(away_pitcher_columns, num)
                away_pitcher_columns[num].append(col)
            except (ValueError, IndexError):
                continue
        elif col.startswith('Home_P_') or col.startswith('Home_SP'):
            try:
                if 'SP' in col:
                    num = 0
                else:
                    num = int(col.split('_')[2])
                ensure_length(home_pitcher_columns, num)
                home_pitcher_columns[num].append(col)
            except (ValueError, IndexError):
                continue

    # Sort each category
    for batter_columns in away_batter_columns:
        sorted_columns.extend(sort_player_columns(batter_columns))
    for batter_columns in home_batter_columns:
        sorted_columns.extend(sort_player_columns(batter_columns))
    for pitcher_columns in away_pitcher_columns:
        sorted_columns.extend(sort_player_columns(pitcher_columns))
    for pitcher_columns in home_pitcher_columns:
        sorted_columns.extend(sort_player_columns(pitcher_columns))
    
    return df[sorted_columns]

# Sort columns in the dataset
sorted_df = sort_columns(df)

# Save the sorted dataset
sorted_df.to_csv('model/masterdata.csv', index=False)

### Filter by Date

In [14]:
# Load the sorted dataset     
sorted_df = pd.read_csv('model/masterdata.csv', low_memory=False)

# Function to filter games based on date range
def filter_games_by_date(df):
    # Convert game_date to datetime
    df['game_date'] = pd.to_datetime(df['game_date'])
    
    # Filter out games before April 5 and after October 5
    filtered_df = df[(df['game_date'].dt.month >= 4) & (df['game_date'].dt.month <= 10) &
                     ((df['game_date'].dt.month != 4) | (df['game_date'].dt.day >= 5)) &
                     ((df['game_date'].dt.month != 10) | (df['game_date'].dt.day <= 5))]
    
    return filtered_df

# Filter the games
filtered_df = filter_games_by_date(sorted_df)

# Save the filtered dataset
filtered_df.to_csv('model/masterdata.csv', index=False)